**meta_train.csv 생성**

1. 구글 드라이브 마운트 & 라이브러리 임포트

In [18]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. 경로 & 모델/폴드 수 설정

In [19]:
input_dir   = "/content/drive/MyDrive/디지털경진대회/val_ensemble_folding/"
output_path = "/content/drive/MyDrive/디지털경진대회/meta_train.csv"

models   = ['gemma', 'qwen', 'xone', 'kanana']
n_folds  = 4

3. 각 모델별로 fold 예측 누적

In [20]:
model_full_dfs = {}

for m in models:
    fold_frames = []

    for fold in range(n_folds):
        fp = os.path.join(input_dir, f"val_{m}_fold{fold}.csv")
        if not os.path.isfile(fp):
            raise FileNotFoundError(fp)

        df = pd.read_csv(fp)

        # 필수 컬럼 확인
        if not {'ID', 'generated', 'label'}.issubset(df.columns):
            raise ValueError(f"[{fp}] 'ID', 'generated', 'label' 컬럼 필요")

        fold_frames.append(df[['ID', 'generated', 'label']])

    # 1. 4개 fold를 세로 방향으로 합치기
    full_df = pd.concat(fold_frames, ignore_index=True)

    # 2. ID 중복이 있으면 오류 (한 샘플당 예측 1개만 존재해야 함)
    if full_df['ID'].duplicated().any():
        dup_ids = full_df.loc[full_df['ID'].duplicated(), 'ID'].unique()[:5]
        raise ValueError(f"[{m}] 중복 ID 존재 예: {dup_ids}")

    # 3. ID 기준 정렬
    full_df.sort_values('ID', inplace=True)
    full_df.reset_index(drop=True, inplace=True)

    # 4. 컬럼 이름 변경: generated -> {model}_pred
    full_df.rename(columns={'generated': f"{m}_pred"}, inplace=True)

    # 5. 전체 개수 검증
    if len(full_df) != n_total:
        raise ValueError(f"[{m}] 행 수 {len(full_df)} ≠ train 행 수 {n_total}")

    model_full_dfs[m] = full_df

print("모든 모델의 fold 예측 누적 완료")

모든 모델의 fold 예측 누적 완료


4.  모델별 DF를 ID 기준으로 inner-merge

In [21]:
meta_df = model_full_dfs[models[0]].copy()

for m in models[1:]:
    meta_df = meta_df.merge(
        model_full_dfs[m][['ID', f"{m}_pred", 'label']],
        on='ID',
        how='inner',
        suffixes=('', '_tmp')
    )
    # label 일관성 확인
    if not (meta_df['label'] == meta_df['label_tmp']).all():
        raise ValueError(f"Label 불일치 발생 – 모델 {m}")
    meta_df.drop(columns=['label_tmp'], inplace=True)

5. 최종 정렬 & 컬럼 순서 지정

In [22]:
meta_df.sort_values('ID', inplace=True)
meta_df.reset_index(drop=True, inplace=True)

pred_cols = [f"{m}_pred" for m in models]
meta_df   = meta_df[['ID'] + pred_cols + ['label']]

6. 점검 & 저장

In [23]:
assert meta_df.isna().sum().sum() == 0, "NaN 값 존재"

meta_df.to_csv(output_path, index=False)
print(f"\n meta_train.csv 저장 완료 : {output_path}")
print(f"shape = {meta_df.shape}  (행 {len(meta_df)}, 열 {len(meta_df.columns)})")
print("\n <preview>")
print(meta_df.head())


 meta_train.csv 저장 완료 : /content/drive/MyDrive/디지털경진대회/meta_train.csv
shape = (121524, 6)  (행 121524, 열 6)

 <preview>
            ID  gemma_pred  qwen_pred  xone_pred  kanana_pred  label
0  FOLD0_00000    1.000000   1.000000   1.000000     1.000000      1
1  FOLD0_00001    0.287109   0.235352   0.283203     0.320312      0
2  FOLD0_00002    1.000000   1.000000   1.000000     1.000000      1
3  FOLD0_00003    0.285156   0.417969   0.289062     0.312500      1
4  FOLD0_00004    0.314453   0.337891   0.296875     0.253906      0


**meta_test.csv 생성**

In [24]:
# 라이브러리 로드
import pandas as pd
import os

1. 경로 & 모델 설정

In [25]:
input_dir = '/content/drive/MyDrive/디지털경진대회/test_ensemble_folding/'

output_path = '/content/drive/MyDrive/디지털경진대회/meta_test.csv'

# 사용 중인 base model 이름 리스트
base_models = ['gemma', 'qwen', 'xone', 'kanana']

2. 각 base model들의 원본 test.csv 추론값을 기반으로 meta_test.csv 생성

In [26]:
# 입력 디렉토리 확인
if not os.path.exists(input_dir):
    raise ValueError(f"입력 디렉토리가 존재하지 않습니다: {input_dir}")

try:
    # 모든 base model에 대해 폴드 예측 결과 불러와 평균 계산
    mean_predictions = {}
    ids = None  # 첫 번째 모델의 fold0 파일 ID를 기준으로 사용

    for model in base_models:
        # 각 모델의 4개 fold 예측 파일 읽기
        fold_preds = []
        for fold in range(4):
            file_name = f"test_{model}_fold{fold}.csv"
            file_path = os.path.join(input_dir, file_name)
            if not os.path.isfile(file_path):
                raise FileNotFoundError(f"예측 결과 파일을 찾을 수 없습니다: {file_path}")

            df = pd.read_csv(file_path)

            # 필요한 컬럼 존재 확인
            if 'ID' not in df.columns or 'generated' not in df.columns:
                raise ValueError(f"파일에 'ID' 또는 'generated' 컬럼이 없습니다: {file_path}")

            # ID 열 일치 확인 (모든 파일의 ID 순서가 동일해야 함)
            if ids is None:
                ids = df['ID']
            else:
                if not ids.equals(df['ID']):
                    raise ValueError(f"{model} 모델의 fold{fold} 예측 파일의 ID 순서가 일치하지 않습니다: {file_path}")

            fold_preds.append(df['generated'])

        # 4개 fold의 예측값 평균 계산
        preds_concat = pd.concat(fold_preds, axis=1)
        mean_pred = preds_concat.mean(axis=1)
        mean_predictions[model] = mean_pred.values

    # 메타 테스트 데이터프레임 생성
    meta_df = pd.DataFrame({'ID': ids})
    for model in base_models:
        col_name = f"pred_mean_{model}"
        meta_df[col_name] = mean_predictions[model]

    # 결과를 CSV로 저장
    meta_df.to_csv(output_path, index=False)
    print(f"meta_test.csv 파일이 저장되었습니다: {output_path}")

except Exception as e:
    print(f"오류 발생: {e}")
    raise

meta_test.csv 파일이 저장되었습니다: /content/drive/MyDrive/디지털경진대회/meta_test.csv


3. meta_test.csv 파일 구조 확인

In [27]:
print("\n Top5 rows of meta_test:")
print(meta_df.head(5))


 Top5 rows of meta_test:
          ID  pred_mean_gemma  pred_mean_qwen  pred_mean_xone  \
0  TEST_0000         0.379236        0.924513        0.465177   
1  TEST_0001         0.393475        0.963943        0.994570   
2  TEST_0002         0.237335        0.258387        0.283300   
3  TEST_0003         0.814698        0.871969        0.985487   
4  TEST_0004         0.904595        0.947946        0.952819   

   pred_mean_kanana  
0          0.273906  
1          0.976941  
2          0.213849  
3          0.977091  
4          0.810718  


**meta model 훈련 및 추론**

1. 경로 설정

In [28]:
meta_train_path = '/content/drive/MyDrive/디지털경진대회/meta_train.csv'
meta_test_path  = '/content/drive/MyDrive/디지털경진대회/meta_test.csv'
submission_path = '/content/drive/MyDrive/디지털경진대회/final_submission.csv'

2. 라이브러리 임포트

In [29]:
import pandas as pd, numpy as np, re, warnings
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
warnings.filterwarnings("ignore")

3. meta_train / meta_test 로드 & 검증 및 logit feature 추가

In [30]:
train = pd.read_csv(meta_train_path)
test  = pd.read_csv(meta_test_path)

assert train['ID'].is_unique and test['ID'].is_unique, "ID 중복"
assert not train.isna().any().any() and not test.isna().any().any(), "NaN 존재"

# test 컬럼명  pred_mean_modelX -> modelX_pred
test = test.rename(columns={c: f"{m.group(1)}_pred"
                            for c in test.columns
                            if (m := re.match(r'^pred_mean_(.+)$', c))})

base_cols = [c for c in train.columns if c not in ('ID', 'label')]

# logit 변환
eps = 1e-6
for col in base_cols:
    train[f"{col}_logit"] = np.log((train[col]+eps)/(1-train[col]+eps))
    test[f"{col}_logit"]  = np.log((test[col] +eps)/(1-test[col] +eps))

feature_cols = base_cols + [f"{c}_logit" for c in base_cols]

X_train = train[feature_cols]
y_train = train['label'].astype(float)
X_test  = test[feature_cols].copy()

assert list(X_test.columns) == feature_cols, "feature 순서/누락"

4. (Meta Model) Logistic Regression Cross Validation / LR 학습 및 추론 진행

In [31]:
SEEDS   = [42, 56, 77, 88, 99]
C_grid  = np.logspace(-4, 2.3, 25)

oof_list, test_list = [], []

for seed in SEEDS:
    pipe = LogisticRegressionCV(
        Cs=C_grid,
        cv=StratifiedKFold(n_splits=7, shuffle=True, random_state=seed),
        solver='lbfgs',
        scoring='roc_auc',
        class_weight='balanced',
        max_iter=4000,
        n_jobs=-1,
        random_state=seed
    )

    pipe.fit(X_train, y_train)
    print(f"seed {seed:<3d} ▸ Best C :", pipe.C_[0])

    oof_pred  = pipe.predict_proba(X_train)[:, 1]
    auc       = roc_auc_score(y_train, oof_pred)
    print(f"          OOF AUC = {auc:.6f}")

    oof_list.append(oof_pred)
    test_list.append(pipe.predict_proba(X_test)[:, 1])

# 평균 앙상블
oof_avg  = np.mean(oof_list,  axis=0)
pred_avg = np.mean(test_list, axis=0)
print("\n OOF AUC (seed ensemble) :", roc_auc_score(y_train, oof_avg))

seed 42  ▸ Best C : 0.02304092976055847
          OOF AUC = 0.770861
seed 56  ▸ Best C : 0.2585234839562191
          OOF AUC = 0.770867
seed 77  ▸ Best C : 0.14125375446227553
          OOF AUC = 0.770866
seed 88  ▸ Best C : 2.900681198693156
          OOF AUC = 0.770878
seed 99  ▸ Best C : 0.042169650342858224
          OOF AUC = 0.770861

 OOF AUC (seed ensemble) : 0.770866689525477


5. 최종 제출 파일 저장

In [32]:
submission = pd.DataFrame({'ID': test['ID'], 'generated': pred_avg})
submission.to_csv(submission_path, index=False)
print(f"\n Saved submission → {submission_path}")
print(submission.head())

assert submission.shape[1] == 2, "컬럼이 일치하지 않습니다."
assert submission['generated'].isna().sum() == 0, "NaN 존재"


 Saved submission → /content/drive/MyDrive/디지털경진대회/final_submission.csv
          ID  generated
0  TEST_0000   0.510497
1  TEST_0001   0.925113
2  TEST_0002   0.263866
3  TEST_0003   0.925431
4  TEST_0004   0.886927


6. 제출 파일 검증 (행 수 확인)

In [33]:
if len(test) == len(submission):
    print(f"행 수 일치: meta_test.csv와 final_submission.csv = {len(test)}")
else:
    print(f"행 수 불일치: meta_test.csv = {len(test)},  final_submission = {len(submission)}")

행 수 일치: meta_test.csv와 final_submission.csv = 1962
